Retrieved index constituent data from:
https://en.wikipedia.org/wiki/S%26P_GSCI
https://en.wikipedia.org/wiki/Bloomberg_Commodity_Index

In [62]:
import eikon as ek
import numpy as np
import pandas as pd
import json
import datetime as dt
import traceback

In [2]:
def initialize_eikon():
    with open("config.json") as f:
        key = json.load(f)["eikon_key"]

    ek.set_app_key(key)

In [33]:
def load_universe():
    """
    Get the universe of futures contracts to trade

    Returns: a dictionary of Eikon continuous futures tickers grouped by the asset subclass

    """
    universe_df = pd.read_csv("universe.csv")

    return_dict = {}
    for c in universe_df.type.unique():
        return_dict[c] = list(universe_df[universe_df.type == c].eikon_ticker)

    return return_dict

In [68]:
def get_price_data(ticker: str, start_date: dt.date, end_date: dt.date):
    """
    Get historical price data on a single contract

    Args:
        ticker: the ticker of the contract to get
        start_date: the start date of the time series
        end_date: the end date of the time series

    Returns: a data frame containing historical price data on the contract

    """
    data = ek.get_data(
        ticker,
        [
            'TR.CLOSEPRICE.date',
            'TR.CLOSEPRICE',
            'TR.ACCUMULATEDVOLUME'
        ],
        {'SDate': start_date.strftime("%Y-%m-%d"), 'Frq': 'D', 'EDate': end_date.strftime("%Y-%m-%d")}
    )

    data = data[0]

    data = data[data.Date != ""].copy()

    data.Date = data.Date.apply(lambda x: dt.date(int(x[0:4]), int(x[5:7]), int(x[8:10])))

    return data

In [40]:
def get_multiple_price_data(universe, start_date, end_date, continuation: int = 1):
    """
    Get a dictionary with historical prices for all futures in the given universe

    :param universe: a dictionary with all the required futures contracts
    :param start_date: the start date to get data from
    :param end_date: the end date to get data from
    :param continuation: the futures continuation to fetc
    :return: a dictionary with historical prices for each of the given futures
    """
    result_collector = dict()

    for asset_class in universe:
        for ticker in universe[asset_class]:
            try:
                ticker = ticker + "c" + str(continuation)
                price_data = get_price_data(ticker, start_date, end_date)
                result_collector[ticker] = price_data
            except:
                continue

    return result_collector

In [45]:
initialize_eikon()
universe = load_universe()

In [35]:
universe

{'Agriculture': ['SB', 'LSU', 'KC', 'ICF', 'CT', 'W', 'KW', 'C', 'S', 'CC'],
 'Base Metals': ['HG', 'SAF', 'SZN', 'SNI', 'SPB'],
 'Energy': ['CL', 'NG', 'LCO', 'LGO', 'RB', 'HO', 'HO', 'LHO'],
 'Livestock': ['LC', 'LH', 'FC'],
 'Precious Metals': ['GC', 'SI']}

In [69]:
price_data_c1 = get_multiple_price_data(
    universe=universe,
    start_date=dt.date(1980, 10, 1),
    end_date=dt.date(2022, 1, 1)
)

In [70]:
len(price_data_c1.keys())

27

In [71]:
pd.concat(price_data_c1.values()).to_csv("price_data_c1.csv", index=False)

In [72]:
price_data_c2 = get_multiple_price_data(
    universe=universe,
    start_date=dt.date(1991, 10, 1),
    end_date=dt.date(2022, 1, 1),
    continuation=2
)

In [73]:
len(price_data_c2.keys())

27

In [74]:
pd.concat(price_data_c2.values()).to_csv("price_data_c2.csv", index=False)

In [75]:
price_data_c3 = get_multiple_price_data(
    universe=universe,
    start_date=dt.date(1991, 10, 1),
    end_date=dt.date(2022, 1, 1),
    continuation=3
)

In [76]:
len(price_data_c3.keys())

27

In [77]:
pd.concat(price_data_c3.values()).to_csv("price_data_c3.csv", index=False)

In [78]:
price_data_c4 = get_multiple_price_data(
    universe=universe,
    start_date=dt.date(1991, 10, 1),
    end_date=dt.date(2022, 1, 1),
    continuation=3
)

In [79]:
len(price_data_c4.keys())

27

In [80]:
pd.concat(price_data_c4.values()).to_csv("price_data_c4.csv", index=False)

In [91]:
t1 = pd.read_csv("price_data_c1.csv")
t2 = pd.read_csv("price_data_c2.csv")
t3 = pd.read_csv("price_data_c3.csv")
t4 = pd.read_csv("price_data_c4.csv")

t1.drop(columns=["Unnamed: 0"], inplace=True)
t2.drop(columns=["Unnamed: 0"], inplace=True)
t3.drop(columns=["Unnamed: 0"], inplace=True)
t4.drop(columns=["Unnamed: 0"], inplace=True)

t1["Continuation"] = "c1"
t2["Continuation"] = "c2"
t3["Continuation"] = "c3"
t4["Continuation"] = "c4"

t1["Instrument"] = t1["Instrument"].apply(lambda x: x[:-2])
t2["Instrument"] = t2["Instrument"].apply(lambda x: x[:-2])
t3["Instrument"] = t3["Instrument"].apply(lambda x: x[:-2])
t4["Instrument"] = t4["Instrument"].apply(lambda x: x[:-2])

In [92]:
test_format_1 = pd.concat([t1, t2, t3, t4])
test_format_1

,Instrument,Date,Close Price,Accumulated Volume,Continuation
0,SB,1980-10-01,NaN,275.0,c1
1,SB,1987-08-31,5.250,204.0,c1
2,SB,1988-12-30,10.000,155.0,c1
3,SB,1989-12-29,12.450,132.0,c1
4,SB,1991-02-28,8.950,137.0,c1
...,...,...,...,...,...
172512,SI,2021-12-27,23.100,111.0,c4
172513,SI,2021-12-28,23.100,160.0,c4
172514,SI,2021-12-29,22.860,45.0,c4
172515,SI,2021-12-30,23.075,109.0,c4


In [105]:
t1 = pd.read_csv("price_data_c1.csv")
t2 = pd.read_csv("price_data_c2.csv")
t3 = pd.read_csv("price_data_c3.csv")
t4 = pd.read_csv("price_data_c4.csv")

t1.drop(columns=["Unnamed: 0"], inplace=True)
t2.drop(columns=["Unnamed: 0"], inplace=True)
t3.drop(columns=["Unnamed: 0"], inplace=True)
t4.drop(columns=["Unnamed: 0"], inplace=True)

t1["Instrument"] = t1["Instrument"].apply(lambda x: x[:-2])
t2["Instrument"] = t2["Instrument"].apply(lambda x: x[:-2])
t3["Instrument"] = t3["Instrument"].apply(lambda x: x[:-2])
t4["Instrument"] = t4["Instrument"].apply(lambda x: x[:-2])

t2.rename(
    columns={
        "Accumulated Volume": "C2 - Accumulated Volume",
        "Close Price": "C2 - Close Price"
    },
    inplace=True
)

t3.rename(
    columns={
        "Accumulated Volume": "C3 - Accumulated Volume",
        "Close Price": "C3 - Close Price"
    },
    inplace=True
)

t4.rename(
    columns={
        "Accumulated Volume": "C4 - Accumulated Volume",
        "Close Price": "C4 - Close Price"
    },
    inplace=True
)

In [108]:
test_format_2 = t1.merge(t2, how="left").merge(t3, how="left").merge(t4, how="left")
test_format_2[~pd.isnull(test_format_2["C2 - Close Price"])]

,Instrument,Date,Close Price,Accumulated Volume,C2 - Close Price,C2 - Accumulated Volume,C3 - Close Price,C3 - Accumulated Volume,C4 - Close Price,C4 - Accumulated Volume
18,SB,1991-10-01,8.880,15.0,8.770,2007.0,8.730,549.0,8.730,549.0
19,SB,1991-10-02,8.850,152.0,8.740,1317.0,8.720,430.0,8.720,430.0
20,SB,1991-10-03,8.550,82.0,8.480,3825.0,8.480,865.0,8.480,865.0
21,SB,1991-10-04,8.660,52.0,8.590,915.0,8.550,74.0,8.550,74.0
22,SB,1991-10-07,8.750,27.0,8.650,1109.0,8.640,110.0,8.640,110.0
...,...,...,...,...,...,...,...,...,...,...
184232,SI,2021-12-27,22.680,1.0,23.060,108793.0,23.100,111.0,23.100,111.0
184233,SI,2021-12-28,23.150,211.0,23.015,73451.0,23.100,160.0,23.100,160.0
184234,SI,2021-12-29,22.635,172.0,22.825,98976.0,22.860,45.0,22.860,45.0
184235,SI,2021-12-30,23.070,344.0,23.100,75359.0,23.075,109.0,23.075,109.0
